Customizations Made

1. Changed dataset from `wikitext-2-raw-v1` to `ag_news` for real-world text variety.  
2. Added `tokenizer.pad_token = tokenizer.eos_token` to handle padding safely.  
3. Fixed `ArrowInvalid` error using `batch_size=None` and `remove_columns` in `.map()`.  
4. Reduced `block_size` from 512 to **128** for efficient small-batch processing.  
5. Added validation check for sequence length (128), created DataLoader, and saved processed dataset.  


In [2]:
!pip install --upgrade pip
!pip install torch transformers datasets tqdm

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached aiohttp-3.13.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.1 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached datasets-4.2.0-py3-none-any.whl (506 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached aiohttp-3.13.1-cp313-cp313-macosx_11_0_arm64.whl (487 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [datasets]5/6 [datasets]ers]


In [3]:
import datasets, transformers, torch, tqdm
print("✅ All libraries imported successfully!")


/Users/dhikshamathan/Desktop/MLOps/MLOps/Labs/Data_Labs/LLM_DATA_PIPELINE/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch


In [5]:
# Load AG News dataset (subset for demonstration)
dataset = load_dataset("ag_news", split="train[:5000]")
print(f"✅ Loaded {len(dataset)} samples.")
print(dataset[0])


Generating test split: 100%|██████████| 7600/7600 [00:00<00:00, 2250385.49 examples/s]

✅ Loaded 5000 samples.
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


In [6]:
# Use GPT-2 tokenizer for consistency with LLM architectures
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 does not have a padding token — set it manually
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # tokenize each text with truncation
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print("✅ Tokenization complete.")
tokenized_ds[0]


Map: 100%|██████████| 5000/5000 [00:00<00:00, 19853.42 examples/s]

✅ Tokenization complete.


{'label': 2,
 'input_ids': [22401,
  520,
  13,
  15682,
  30358,
  5157,
  20008,
  262,
  2619,
  357,
  12637,
  8,
  8428,
  532,
  10073,
  12,
  7255,
  364,
  11,
  5007,
  3530,
  338,
  45215,
  59,
  3903,
  286,
  14764,
  12,
  948,
  77,
  873,
  11,
  389,
  4379,
  4077,
  757,
  13],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [10]:
block_size = 128

def group_texts(examples):
    concatenated = sum(examples["input_ids"], [])
    total_length = (len(concatenated) // block_size) * block_size
    concatenated = concatenated[:total_length]

    input_blocks = [concatenated[i:i+block_size] for i in range(0, total_length, block_size)]
    return {"input_ids": input_blocks, "labels": input_blocks.copy()}

# ✅ Final working version (resolves ArrowInvalid)
lm_dataset = tokenized_ds.map(
    group_texts,
    batched=True,
    batch_size=None,
    remove_columns=tokenized_ds.column_names,  # <- critical line
    load_from_cache_file=False
)

print(f"✅ Created {len(lm_dataset)} grouped sequences.")



Map: 100%|██████████| 5000/5000 [00:02<00:00, 1973.90 examples/s]

✅ Created 2155 grouped sequences.


In [11]:
# ✅ Sanity check: ensure every grouped sequence has length = 128
lengths = [len(x["input_ids"]) for x in lm_dataset]
unique_lengths = set(lengths)
print(f"✅ Unique sequence lengths: {unique_lengths}")
print(f"Total sequences checked: {len(lengths)}")


✅ Unique sequence lengths: {128}
Total sequences checked: 2155


In [12]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.tensor([x["input_ids"] for x in batch], dtype=torch.long)
    labels = input_ids.clone()
    return {"input_ids": input_ids, "labels": labels}

train_loader = DataLoader(lm_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Inspect one batch
for batch in train_loader:
    print("✅ Batch tensor shape:", batch["input_ids"].shape)
    break


✅ Batch tensor shape: torch.Size([8, 128])


In [13]:
import os
os.makedirs("data", exist_ok=True)
lm_dataset.save_to_disk("data/ag_news_lm_tokenized")
print("💾 Saved dataset to data/ag_news_lm_tokenized/")


Saving the dataset (1/1 shards): 100%|██████████| 2155/2155 [00:00<00:00, 201420.06 examples/s]

💾 Saved dataset to data/ag_news_lm_tokenized/
